# Project Set-up


In [1]:
try: 
    import pandas as pd
    import ta
    import numpy as np
    import requests
    import sys, os
    from dotenv import load_dotenv
    from pathlib import Path
    from datetime import datetime, timedelta
    
    print("Successfully Imported all the libraries")

except Exception as e: 
    print(f"Import Error: {e}")
    raise


Successfully Imported all the libraries


In [2]:
# PROJECT_ROOT = Path('/Users/shengyaotang/Desktop/Quant Project/ML_Portfolio_Management')
PROJECT_ROOT = Path('..')
DATA_DIR = PROJECT_ROOT/'data'

load_dotenv(PROJECT_ROOT/'.env')
print(".env file successfully loaded")


.env file successfully loaded


# Data Extraction: 
Extract S&P 500 healthcare data from wikipedia and financial modeling Prep Api

### Get Ticker data

In [3]:

alpha_vantage_api_key = os.getenv("ALPHA_VANTAGE_API_KEY")
etf_symbol = "DIA"

def get_qqq_holdings(api_key: str, symbol: str) -> pd.DataFrame:

    url = f'https://www.alphavantage.co/query?function=ETF_PROFILE&symbol={symbol}&apikey={api_key}'
    response = requests.get(url)
    
    if response.status_code == 200: 
        json_data = response.json()
        holdings_data = json_data["holdings"]
        df = pd.DataFrame(holdings_data)
        rows_to_drop = df[df['symbol'] == 'n/a'].index
        df = df.drop(rows_to_drop, axis = 0)

    else: 
        print("API request to Alpha Vantage failed.")
    df.to_csv(DATA_DIR/"raw"/"holdings.csv", index = False)
    # df.to_csv('../data/raw/holdings.csv', index = False)
    return df


In [4]:
df = get_qqq_holdings(alpha_vantage_api_key, etf_symbol)
holdings_list = df['symbol'].to_list()
print(holdings_list)

['GS', 'CAT', 'MSFT', 'AXP', 'V', 'HD', 'UNH', 'SHW', 'JPM', 'AMGN', 'IBM', 'MCD', 'TRV', 'CRM', 'AAPL', 'AMZN', 'BA', 'HON', 'JNJ', 'NVDA', 'MMM', 'CVX', 'PG', 'DIS', 'WMT', 'MRK', 'CSCO', 'KO', 'NKE', 'VZ']


### Extract historical price data from financial modeling prep

In [5]:
base_url = "https://financialmodelingprep.com"
data_type = "historical-price-eod"
ticker = "AAPL"
FMP_API_KEY = os.getenv("FMP_API_KEY")

url = f"{base_url}/stable/{data_type}/full?symbol={ticker}&apikey={FMP_API_KEY}"


In [6]:
def get_historical_price(url: str) -> pd.DataFrame: 
    try: 
        response = requests.get(url)
        print(response.status_code)
        data = response.json()
        df = pd.DataFrame(data, index = range(len(data)))
        print("Got historical price successfully")
    except Exception as e:
        # print(f"There is an error: {e}")
        df = pd.DataFrame()
        
    return df

In [7]:
def get_historical_prices(tickers: list, api_key: str) -> pd.DataFrame:

    df = pd.DataFrame()
    for ticker in tickers:
        url = f"{base_url}/stable/{data_type}/full?symbol={ticker}&apikey={api_key}"
        temp_df = get_historical_price(url)
        df = pd.concat([df,temp_df], axis = 0)

    # df.to_csv("/Users/shengyaotang/Desktop/Quant Project/ML_Portfolio_Management/data/raw/historical_price.csv", index = False)
    df.to_csv(DATA_DIR/"raw"/"historical_price.csv", index = False)
    # df.to_csv('../data/raw/historical_price.csv', index = False)
    return df

In [8]:
df = get_historical_prices(holdings_list, FMP_API_KEY)

200
Got historical price successfully
402
200
Got historical price successfully
200
Got historical price successfully
402
200
Got historical price successfully
402
200
Got historical price successfully
402
402
402
402
402
200
Got historical price successfully
200
Got historical price successfully
200
Got historical price successfully
402
200
Got historical price successfully
200
Got historical price successfully
402
200
Got historical price successfully
402
200
Got historical price successfully
200
Got historical price successfully
402
200
Got historical price successfully
200
Got historical price successfully
200
Got historical price successfully
200
Got historical price successfully


In [9]:
df.head(10)
df_holdings = pd.DataFrame(df["symbol"].unique(), columns = ["symbol"])
df_holdings.to_csv(DATA_DIR/"raw"/"holdings.csv", index = False)



# Data Transformation

In [10]:
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin

from ta.momentum import RSIIndicator
from ta.volatility import BollingerBands

### Import the dataset

In [11]:
def load_data(filename): 
    df = pd.read_csv(filename)
    return df

In [12]:
# df = read_data(DATA_DIR)
df = load_data(DATA_DIR/"raw"/"historical_price.csv")
df

,symbol,date,open,high,low,close,volume,change,changePercent,vwap
0,GS,2026-01-15,924.90,981.26,924.67,975.86,3773730,50.96,5.510000,951.6725
1,GS,2026-01-14,932.30,938.15,917.90,932.67,2513038,0.37,0.039687,930.2550
2,GS,2026-01-13,947.32,949.94,931.00,938.15,1969600,-9.17,-0.967990,941.6025
3,GS,2026-01-12,934.00,950.56,929.11,949.55,2180033,15.55,1.660000,940.8050
4,GS,2026-01-09,938.77,946.14,932.70,938.98,1334100,0.21,0.022370,939.1475
...,...,...,...,...,...,...,...,...,...,...
21330,VZ,2021-01-25,57.47,58.62,57.18,58.42,22444344,0.95,1.650000,57.9225
21331,VZ,2021-01-22,57.17,57.69,56.76,57.47,16364509,0.30,0.524750,57.2725
21332,VZ,2021-01-21,57.19,57.51,57.05,57.27,13604851,0.08,0.139880,57.2550
21333,VZ,2021-01-20,57.01,57.39,56.66,57.26,19923100,0.25,0.438520,57.0800


### Create Target Variable
1. Calculate weekly log returns

In [13]:
# Calculate weekly returns:
def calculate_weekly_returns(df: pd.DataFrame) -> pd.DataFrame:
    df['weekly_return'] = df.groupby('symbol')['close'].pct_change(periods = -5)
    df = df.dropna(subset = ["weekly_return"])
    
    df["weekly_log_return"] = np.log(1 + df["weekly_return"])
    
    return df

In [14]:
df = calculate_weekly_returns(df)
df.head(3)

/var/folders/gl/wt1cfyrs16358nbd8jfg3k700000gn/T/ipykernel_1660/387843821.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["weekly_log_return"] = np.log(1 + df["weekly_return"])


,symbol,date,open,high,low,close,volume,change,changePercent,vwap,weekly_return,weekly_log_return
0,GS,2026-01-15,924.90,981.26,924.67,975.86,3773730,50.96,5.510000,951.6725,0.043890,0.042954
1,GS,2026-01-14,932.30,938.15,917.90,932.67,2513038,0.37,0.039687,930.2550,-0.008873,-0.008913
2,GS,2026-01-13,947.32,949.94,931.00,938.15,1969600,-9.17,-0.967990,941.6025,-0.018127,-0.018294


2. 🎯 Create Target Variable

In [15]:
def create_target_variable(df: pd.DataFrame) -> pd.DataFrame:
    
    df = df.copy()
    
    df['date'] = pd.to_datetime(df['date'])
    df['year_week'] = df['date'].dt.strftime('%Y-%U')
    
    median_returns = df.groupby('year_week')['weekly_log_return'].median().reset_index()
    median_returns.columns = ['year_week', 'median_weekly_return']
    
    # Merge median returns back to original dataframe
    if 'median_weekly_return' not in df.columns:
        df = pd.merge(df, median_returns, on='year_week', how='left')
        
    df['target'] = (df['weekly_log_return'] > df['median_weekly_return']).astype(int)
    
    return df

In [16]:
df = create_target_variable(df)
df.head(3)

,symbol,date,open,high,low,close,volume,change,changePercent,vwap,weekly_return,weekly_log_return,year_week,median_weekly_return,target
0,GS,2026-01-15,924.90,981.26,924.67,975.86,3773730,50.96,5.510000,951.6725,0.043890,0.042954,2026-02,-0.004291,1
1,GS,2026-01-14,932.30,938.15,917.90,932.67,2513038,0.37,0.039687,930.2550,-0.008873,-0.008913,2026-02,-0.004291,0
2,GS,2026-01-13,947.32,949.94,931.00,938.15,1969600,-9.17,-0.967990,941.6025,-0.018127,-0.018294,2026-02,-0.004291,0


In [17]:
# 🗂 Create the panel dataset indexed by date and tickers

def create_panel_dataset(df: pd.DataFrame) -> pd.DataFrame:
    return df.sort_values(by = "date")

In [18]:
# This step could be moved to the pipeline
df = create_panel_dataset(df)
df.head(3)

,symbol,date,open,high,low,close,volume,change,changePercent,vwap,weekly_return,weekly_log_return,year_week,median_weekly_return,target
21249,VZ,2021-01-26,56.99,57.28,56.41,56.57,37892724,-0.42000,-0.73697,56.8125,-0.008414,-0.008449,2021-04,-0.031718,1
1249,GS,2021-01-26,283.93,285.77,280.08,281.76,3296846,-2.17000,-0.76427,282.8850,-0.042284,-0.043204,2021-04,-0.031718,0
16249,WMT,2021-01-26,48.54,49.27,48.49,49.17,15031500,0.62667,1.30000,48.8675,0.028661,0.028258,2021-04,-0.031718,1


In [19]:
# 🗑 Remove columns that will not be used for the project
def remove_columns(cols: list, df: pd.DataFrame) -> pd.DataFrame:
    try: 
        df = df.drop(columns = cols, axis = 1)
    except Exception as e:
        print(e)
    finally: 
        return df

In [20]:
cols = ["open", "high", "low", "change", 
        "changePercent", "weekly_return", 
       "year_week", "median_weekly_return"]
df = remove_columns(cols, df)
df.head(3)

,symbol,date,close,volume,vwap,weekly_log_return,target
21249,VZ,2021-01-26,56.57,37892724,56.8125,-0.008449,1
1249,GS,2021-01-26,281.76,3296846,282.8850,-0.043204,0
16249,WMT,2021-01-26,49.17,15031500,48.8675,0.028258,1


2. Calculate moving averages

In [21]:
#📈 Moving Averages

def calc_moving_avg(window: int, df: pd.DataFrame) -> pd.DataFrame:
    col_name = f'MA_{str(window)}'
    df[col_name] = df.groupby('symbol')['close'].transform(
        lambda x: x.rolling(window = window, min_periods = 1).mean()
    )
    
    return df


In [22]:
def calc_moving_avgs(windows: list, df: pd.DataFrame) -> pd.DataFrame: 
    for window in windows: 
        df = calc_moving_avg(window, df)
    return df

In [23]:
windows = [200, 100, 50]
df = calc_moving_avgs(windows, df)
df.head(3)

,symbol,date,close,volume,vwap,weekly_log_return,target,MA_200,MA_100,MA_50
21249,VZ,2021-01-26,56.57,37892724,56.8125,-0.008449,1,56.57,56.57,56.57
1249,GS,2021-01-26,281.76,3296846,282.8850,-0.043204,0,281.76,281.76,281.76
16249,WMT,2021-01-26,49.17,15031500,48.8675,0.028258,1,49.17,49.17,49.17


### Risk and Volatility

In [24]:
def calc_volatility(col_name: str, window: int, df: pd.DataFrame) -> pd.DataFrame: 
    df[col_name] = df.groupby('symbol')['weekly_log_return'].transform(
        lambda x: x.rolling(window = window, min_periods = 1).std() * np.sqrt(window)
    )
    
    return df

In [25]:
def calc_volatilties(volatility_dict: dict, df: pd.DataFrame) -> pd.DataFrame: 
    for col_name, window in volatility_dict.items():
        df = calc_volatility(col_name, window, df)
    return df

In [26]:
volatility_dict = {
    'vol_1M': 4,    # 1 month ≈ 4 weeks
    'vol_6M': 26,   # 6 months ≈ 26 weeks
    'vol_12M': 52   # 12 months ≈ 52 weeks
}

df = calc_volatilties(volatility_dict, df)
#df = df.dropna(subset = ['vol_1M', 'vol_6M','vol_12M'])

df.head(3)

,symbol,date,close,volume,vwap,weekly_log_return,target,MA_200,MA_100,MA_50,vol_1M,vol_6M,vol_12M
21249,VZ,2021-01-26,56.57,37892724,56.8125,-0.008449,1,56.57,56.57,56.57,NaN,NaN,NaN
1249,GS,2021-01-26,281.76,3296846,282.8850,-0.043204,0,281.76,281.76,281.76,NaN,NaN,NaN
16249,WMT,2021-01-26,49.17,15031500,48.8675,0.028258,1,49.17,49.17,49.17,NaN,NaN,NaN


### Short Term Reversal Factors

In [27]:
def calculate_rsi(window, group): 
    rsi = RSIIndicator(close = group['close'], window = window).rsi()
    return rsi

In [28]:
def calculate_rsis(windows: list, df: pd.DataFrame) -> pd.DataFrame: 
    for window in windows: 
        col_name = f'RSI_{str(window)}'
        group = df.groupby('symbol', group_keys = False)
        df[col_name] = group.apply(lambda x: calculate_rsi(window, x))
        
    return df

In [29]:
windows = [3, 9, 14]
df = calculate_rsis(windows, df)
df.head(3)

/var/folders/gl/wt1cfyrs16358nbd8jfg3k700000gn/T/ipykernel_1660/1543531198.py:5: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df[col_name] = group.apply(lambda x: calculate_rsi(window, x))
/var/folders/gl/wt1cfyrs16358nbd8jfg3k700000gn/T/ipykernel_1660/1543531198.py:5: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df[col_name] = group.apply(lambda x: calculate_rsi(window, x))
/var/folders/gl/wt1cfyrs16358nbd8jfg3k70

,symbol,date,close,volume,vwap,weekly_log_return,target,MA_200,MA_100,MA_50,vol_1M,vol_6M,vol_12M,RSI_3,RSI_9,RSI_14
21249,VZ,2021-01-26,56.57,37892724,56.8125,-0.008449,1,56.57,56.57,56.57,NaN,NaN,NaN,NaN,NaN,NaN
1249,GS,2021-01-26,281.76,3296846,282.8850,-0.043204,0,281.76,281.76,281.76,NaN,NaN,NaN,NaN,NaN,NaN
16249,WMT,2021-01-26,49.17,15031500,48.8675,0.028258,1,49.17,49.17,49.17,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
def calculate_bb(group, window = 20, window_dev = 2):
    return BollingerBands(close = group['close'], window = window, window_dev = window_dev)

In [31]:
def calculate_bbs(bands: list, df:pd.DataFrame) -> pd.DataFrame:
    group = df.groupby('symbol', group_keys = False)
    for band in bands:
        if band == "hband":
            df[band] = group.apply(lambda x: calculate_bb(x).bollinger_hband())
        elif band == "lband":
            df[band] = group.apply(lambda x: calculate_bb(x).bollinger_lband())
            
    return df

In [32]:
bands = ["hband", "lband"]
df = calculate_bbs(bands, df)
df.head(3)

/var/folders/gl/wt1cfyrs16358nbd8jfg3k700000gn/T/ipykernel_1660/3629412943.py:5: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df[band] = group.apply(lambda x: calculate_bb(x).bollinger_hband())
/var/folders/gl/wt1cfyrs16358nbd8jfg3k700000gn/T/ipykernel_1660/3629412943.py:7: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df[band] = group.apply(lambda x: calculate_bb(x).bollinger_lband())


,symbol,date,close,volume,vwap,weekly_log_return,target,MA_200,MA_100,MA_50,vol_1M,vol_6M,vol_12M,RSI_3,RSI_9,RSI_14,hband,lband
21249,VZ,2021-01-26,56.57,37892724,56.8125,-0.008449,1,56.57,56.57,56.57,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1249,GS,2021-01-26,281.76,3296846,282.8850,-0.043204,0,281.76,281.76,281.76,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16249,WMT,2021-01-26,49.17,15031500,48.8675,0.028258,1,49.17,49.17,49.17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Momentum Factor: 

In [33]:
def calculate_momentum(window_months: int, df: pd.DataFrame, trading_days :int = 21) -> pd.DataFrame:
    window_days = window_months * trading_days
    col_name = f"momentum_{str(window_months)}M"
    df[col_name] = df.groupby('symbol')['close'].pct_change(periods = window_days)
    
    return df

In [34]:
def calculate_momentums(windows: list, df:pd.DataFrame) -> pd.DataFrame:
    for window in windows:
        df = calculate_momentum(window, df)
    return df

In [35]:
windows_for_momentum = [12, 6, 1]

df = calculate_momentums(windows_for_momentum, df)
df.head(3)

,symbol,date,close,volume,vwap,weekly_log_return,target,MA_200,MA_100,MA_50,...,vol_6M,vol_12M,RSI_3,RSI_9,RSI_14,hband,lband,momentum_12M,momentum_6M,momentum_1M
21249,VZ,2021-01-26,56.57,37892724,56.8125,-0.008449,1,56.57,56.57,56.57,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1249,GS,2021-01-26,281.76,3296846,282.8850,-0.043204,0,281.76,281.76,281.76,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16249,WMT,2021-01-26,49.17,15031500,48.8675,0.028258,1,49.17,49.17,49.17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Filter Datasets:

In [36]:
# 🗓Choose Each Wednesday and Thursday as the data transformation
def wed_thurs_selector(df, date_col='date', stock_col='symbol'):

    df['year_week'] = df[date_col].dt.strftime('%Y-%U')
    df['day_of_week'] = df[date_col].dt.dayofweek
    
    
    wed_thu = df[df['day_of_week'].isin([2, 3])].copy()
    
    
    filtered = wed_thu.groupby([stock_col, 'year_week']).first().reset_index()
    
    return filtered.drop(columns=['day_of_week', 'year_week'])
    

In [37]:
def filter_data(df: pd.DataFrame) -> pd.DataFrame: 
    df = df.copy()
    df = wed_thurs_selector(df)
    return df.dropna(ignore_index = True)

In [38]:
df = filter_data(df)
df.head(3)

,symbol,date,close,volume,vwap,weekly_log_return,target,MA_200,MA_100,MA_50,...,vol_6M,vol_12M,RSI_3,RSI_9,RSI_14,hband,lband,momentum_12M,momentum_6M,momentum_1M
0,AAPL,2022-01-26,159.69,108275308,161.35,-0.040138,0,147.80170,158.1554,169.0138,...,0.179705,0.298390,3.345620,22.148664,32.126858,184.839828,157.965172,0.124102,0.101462,-0.114457
1,AAPL,2022-02-02,175.84,84914300,174.95,0.096340,1,148.73240,159.0118,170.6512,...,0.245340,0.330057,90.620084,65.943436,59.887269,180.925805,157.671195,0.312827,0.196597,-0.033899
2,AAPL,2022-02-09,176.28,71285038,175.97,0.002499,1,149.72835,160.2769,171.9840,...,0.249891,0.315592,79.428507,64.007748,59.341041,181.167717,157.599283,0.302016,0.208556,0.023753


In [39]:
def save_processed_data(df: pd.DataFrame, directory): 
    df.to_csv(directory/"processed"/"processed_historical_price.csv", index = False)

In [40]:
save_processed_data(df, DATA_DIR)

### Create Custom transformer
E.g. Remove/Fill in Nan values, log transformation, standardization

In [41]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd

In [42]:
# Perform Log Transformation: 
class LogTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, features):
        self.features = features

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        for feature in self.features:
            X[feature] = np.log(X['close'] / X[feature])
        return X

### 💽Create training and testing variable

In [43]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import KFold

import joblib

In [44]:
# Create x and y variable

def create_variables(df: pd.DataFrame) -> list: 
    return [df.drop(columns = ["target"]), df["target"]]

X, y = create_variables(df)


In [45]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [46]:
# symbols = df["symbol"].unique().tolist()

# X_train, X_test, y_train, y_test = pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame()
# for symbol in symbols: 
#     symbol_df = df[df["symbol"] == symbol]
#     cut = int(len(symbol_df) * 0.8)
#     Xs, ys = create_variables(symbol_df)
#     Xs_train, Xs_test = Xs[:cut], Xs[cut:]
#     ys_train, ys_test = ys[:cut], ys[cut:]

#     X_train = pd.concat([X_train, Xs_train])
#     X_test = pd.concat([X_test, Xs_test])
#     y_train = pd.concat([y_train, ys_train])
#     y_test = pd.concat([y_test, ys_test])


def time_aware_split(df): 
    symbols = df["symbol"].unique().tolist()

    X_train, X_test, y_train, y_test = pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame()
    for symbol in symbols:
        symbol_df = df[df["symbol"] == symbol]
        cut = int(len(symbol_df) * 0.8)
        Xs, ys = create_variables(symbol_df)
        Xs_train, Xs_test = Xs[:cut], Xs[cut:]
        ys_train, ys_test = ys[:cut], ys[cut:]

        X_train = pd.concat([X_train, Xs_train])
        X_test = pd.concat([X_test, Xs_test])
        y_train = pd.concat([y_train, ys_train])
        y_test = pd.concat([y_test, ys_test])

    return X_train, X_test, y_train, y_test



In [47]:
X_train, X_test, y_train, y_test = time_aware_split(df)

In [48]:
# tscv = TimeSeriesSplit(n_splits=3)
# for i, (train_index, test_index) in enumerate(tscv.split(df)):
#     print(f"Fold {i + 1}:")
#     print(f"  Train: index={train_index}")
#     print(f"  Test:  index={test_index}")

### 🏭 Build a pipeline to transform features

In [49]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

In [50]:
#build pipeline
log_transform_features = ['MA_200', 'MA_100', 'MA_50', 'hband', 'lband']
# cols_to_drop = ['Ticker', 'date', 'weekly_log_return']

cols_to_drop = ['symbol', 'date', 'weekly_log_return']

# Example: Drop columns 'col1' and 'col2'
col_dropper = ColumnTransformer(
    transformers=[
        ('drop_cols', 'drop', cols_to_drop)
    ],
    remainder='passthrough'  # keeps all other columns
)

logistic_regression_pipeline = Pipeline(steps = [ 
    ('log_transformers', LogTransformer(log_transform_features)),
    ('col_dropper', col_dropper), 
    ('scaler', StandardScaler()), 
    ('classifier', LogisticRegression())
])

In [51]:
logistic_regression_model = logistic_regression_pipeline.fit(X_train, y_train)

/Users/shengyaotang/opt/anaconda3/envs/bootcamp_env/lib/python3.10/site-packages/sklearn/utils/validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [52]:
filename = "logistic_regression_model.sav"
joblib.dump(logistic_regression_model, open(filename, 'wb'))

In [53]:
logistic_regression_model = joblib.load(open(filename, 'rb'))
y_pred = logistic_regression_model.predict(X_test)
print(y_pred)

[1. 1. 0. 1. 1. 0. 1. 0. 0. 1. 0. 0. 1. 1. 1. 1. 1. 0. 1. 1. 0. 1. 1. 0.
 1. 1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 1. 0. 0. 1. 0. 0. 0. 1. 1. 0. 1. 1. 1.
 1. 0. 1. 1. 1. 0. 0. 1. 1. 0. 1. 0. 1. 1. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0.
 1. 1. 0. 0. 1. 1. 1. 0. 1. 0. 1. 0. 0. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 0.
 0. 1. 1. 1. 1. 0. 1. 1. 0. 1. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 0. 1. 0. 1. 1. 0. 1. 0. 1. 1.
 1. 0. 0. 1. 0. 1. 1. 0. 1. 1. 1. 1. 0. 0. 1. 1. 0. 1. 1. 0. 1. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 1. 1. 0. 1. 0. 1. 1. 0. 1. 0. 1.
 1. 1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 1. 1. 1. 1. 0. 1. 0. 1. 0. 1. 1. 1.
 1. 0. 1. 1. 1. 1. 1. 1. 0. 0. 1. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1.
 0. 0. 1. 0. 0. 1. 1. 1. 1. 0. 0. 0. 1. 1. 0. 1. 1. 0. 1. 0. 1. 0. 1. 1.
 1. 1. 0. 0. 1. 1. 1. 1. 0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 1. 1. 1. 0. 1. 1.
 1. 0. 1. 0. 1. 0. 0. 0. 1. 0. 1. 1. 0. 1. 0. 0. 1. 0. 0. 1. 1. 1. 1. 0.
 1. 1. 1. 0. 1. 0. 0. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1.

In [54]:
print(f"Model accuracy: {logistic_regression_model.score(X_test, y_test)}")

Model accuracy: 0.7661064425770309


# Create Weekly Stock Portfolio

In [55]:
def get_last_week_data(filename):
    df = load_data(filename)
    df['date'] = pd.to_datetime(df['date'])
    df['year_week'] = df['date'].dt.strftime('%Y-%U')
    today = datetime.today()
    target_weekday = 2
    days_ago = today.weekday() - target_weekday + 7
    last_weds_date = (today - timedelta(days = days_ago)).strftime('%Y-%m-%d')
    last_thurs_date = (today - timedelta(days = days_ago - 1)).strftime('%Y-%m-%d')
    # last_year_week = (datetime.now() - timedelta(weeks = 1)).strftime('%Y-%U')

    # df_last_week = df[df['year_week'] == last_year_week]
    df_last_week = df[df['date'] == last_weds_date]
    if len(df_last_week) == 0:
        df_last_week = df[df['date'] == last_thurs_date]
    return df_last_week.drop(columns=['year_week'])

In [56]:
filename = DATA_DIR/'processed'/'processed_historical_price.csv'
df_last_week = get_last_week_data(filename).reset_index(drop = True)
X_last_week = df_last_week.drop(columns = ['target'])
y_last_week = df_last_week['target']


In [57]:
y_pred = logistic_regression_model.predict(X_last_week)
y_prob = logistic_regression_model.predict_proba(X_last_week)

df_pred = pd.DataFrame(y_pred, columns = ['prediction'])
df_prob = pd.DataFrame(y_prob, columns = ['underperform', 'outperform'])

df_pred

,prediction
0,0.0
1,1.0
2,1.0
3,0.0
4,0.0
5,0.0
6,1.0
7,1.0
8,0.0
9,0.0


In [58]:
def create_weekly_stock_portfolio(*args: pd.DataFrame) -> pd.DataFrame:
    df_stock_portfolio = pd.concat(args, axis = 1)
    df_stock_portfolio = df_stock_portfolio.sort_values(by = ['outperform'], ascending = False)
    df_stock_portfolio = df_stock_portfolio[df_stock_portfolio['prediction'] == 1]
    df_stock_portfolio['weight'] = 1 / len(df_stock_portfolio)
    df_stock_portfolio.to_csv(DATA_DIR/'processed'/'stock_portfolio.csv', index = False)
    return df_stock_portfolio.reset_index(drop = True)

In [59]:
df_stock_portfolio = create_weekly_stock_portfolio(df_last_week['symbol'], df_prob, df_pred)
df_stock_portfolio

,symbol,underperform,outperform,prediction,weight
0,AMZN,0.058454,0.941546,1.0,0.125
1,BA,0.154700,0.845300,1.0,0.125
2,NVDA,0.203429,0.796571,1.0,0.125
3,GS,0.219218,0.780782,1.0,0.125
4,MSFT,0.302058,0.697942,1.0,0.125
5,UNH,0.369037,0.630963,1.0,0.125
6,JNJ,0.373734,0.626266,1.0,0.125
7,V,0.377394,0.622606,1.0,0.125


In [60]:
def update_stock_portfolio(df_stock_portfolio: pd.DataFrame, df_historical_price: pd.DataFrame) -> pd.DataFrame:
    df_historical_price = df_historical_price.copy()
    
    df_historical_price['date'] = pd.to_datetime(df_historical_price['date'])
    df_historical_price['week_of_year'] = df_historical_price['date'].dt.strftime('%Y-%U')
    df_weekly_price = df_historical_price[df_historical_price['week_of_year'] == datetime.now().strftime('%Y-%U') ]

    if 'close' not in df_stock_portfolio.columns: 
        df_weekly_portfolio = pd.merge(df_stock_portfolio, df_weekly_price, on = 'symbol', how = 'left')
    df_weekly_portfolio['week_of_day'] = df_weekly_portfolio['date'].dt.weekday
    df_weekly_portfolio.to_csv(DATA_DIR/'processed'/'weekly_portfolio.csv')

    return df_weekly_portfolio.dropna()

In [80]:
df_historical_price = load_data(DATA_DIR/'raw'/'historical_price.csv')
df_weekly_portfolio = update_stock_portfolio(df_stock_portfolio, df_historical_price)
# [1 / df_weekly_portfolio['symbol'].nunique() ] 
df_weekly_portfolio

,symbol,underperform,outperform,prediction,weight,date,open,high,low,close,volume,change,changePercent,vwap,week_of_year,week_of_day
0,AMZN,0.058454,0.941546,1.0,0.125,2026-01-15,239.31,240.65,236.63,238.18,43003571,-1.130,-0.472190,238.6925,2026-02,3
1,AMZN,0.058454,0.941546,1.0,0.125,2026-01-14,241.15,241.28,236.22,236.65,41410600,-4.500,-1.870000,238.8250,2026-02,2
2,AMZN,0.058454,0.941546,1.0,0.125,2026-01-13,246.53,247.66,240.25,242.60,38371800,-3.930,-1.590000,244.2600,2026-02,1
3,AMZN,0.058454,0.941546,1.0,0.125,2026-01-12,246.73,248.94,245.96,246.47,35867800,-0.260,-0.105380,247.0250,2026-02,0
4,BA,0.154700,0.845300,1.0,0.125,2026-01-15,244.38,248.75,243.97,247.74,6517157,3.360,1.370000,246.2100,2026-02,3
5,BA,0.154700,0.845300,1.0,0.125,2026-01-14,244.44,244.89,239.60,242.61,7748606,-1.830,-0.748650,242.8850,2026-02,2
6,BA,0.154700,0.845300,1.0,0.125,2026-01-13,241.12,247.40,240.10,244.55,11460515,3.430,1.420000,243.2925,2026-02,1
7,BA,0.154700,0.845300,1.0,0.125,2026-01-12,234.29,239.95,234.00,239.81,7894000,5.520,2.360000,237.0125,2026-02,0
8,NVDA,0.203429,0.796571,1.0,0.125,2026-01-15,186.50,189.70,186.33,187.05,206188642,0.550,0.294910,187.3950,2026-02,3
9,NVDA,0.203429,0.796571,1.0,0.125,2026-01-14,184.32,184.46,180.80,183.14,159586135,-1.180,-0.640190,183.1800,2026-02,2


In [62]:
def calculate_portfolio_metrics(portfolio: pd.DataFrame) -> pd.DataFrame:
    s = portfolio.groupby("date")['close'].sum()
    df = pd.DataFrame(s, columns=['close']).reset_index()
    df['daily_return'] = df['close'].pct_change()
    df.loc[0, 'daily_return'] = 0
    df['cumulative_return'] = (df['close'] - df['close'][0]) / df['close'][0] 

    df = df.rename(columns = {'close':'total_value'})
    

    return df

# s_portfolio_metric = df_weekly_portfolio.groupby("date")['close'].sum()
# df_portfolio_metric = pd.DataFrame(s_portfolio_metric, columns=['close']).reset_index()

In [63]:
df_weekly_perf = calculate_portfolio_metrics(df_weekly_portfolio)
df_weekly_perf


,date,total_value,daily_return,cumulative_return
0,2026-01-12,2991.38,0.000000,0.000000
1,2026-01-13,2957.24,-0.011413,-0.011413
2,2026-01-14,2937.13,-0.006800,-0.018135
3,2026-01-15,2991.82,0.018620,0.000147


In [64]:
#create a new function that tracks the historical performance. 
#create the new function by merging the calculate weekly portfolio metrics function
def historical_performance(hist_perf_file_path, df_weekly_perf, prev_date: datetime, start_date = "2026-01-12", init_value = 100000):
    if hist_perf_file_path.is_file():
        df = pd.read_csv(hist_perf_file_path)
    else: 
        df = pd.DataFrame()
    df_daily_perf = df_weekly_perf[df_weekly_perf['date'] == pd.Timestamp(prev_date)]

    if len(df) == 0:
        df = pd.concat([df, df_daily_perf], axis = 0)
    else:
        if len(df[df['date'] == str(prev_date)]) == 0:
            df = pd.concat([df, df_daily_perf], axis = 0)

    df = df.reset_index(drop = True)
    df['date'] = pd.to_datetime(df['date'])
    df.loc[0,'total_value'] = init_value
    
    #Calculate the total value
    df['temp_total_value'] = df['total_value'].shift(1)
    df.loc[1:,'total_value'] = df['temp_total_value'] * (1 + df['daily_return'])



    #Calculate the cumulative return iteratively
    df['temp_return'] = df['cumulative_return'].shift(1)
    df.loc[1:,'cumulative_return'] = (1 + df['daily_return'])*(1 + df['temp_return']) - 1

    df = df.drop(columns = ['temp_return', 'temp_total_value'])

    
    
    df.to_csv(hist_perf_file_path, index = False)

    

    return df

In [65]:
hist_perf_file_path = DATA_DIR/'processed'/'historical_performance.csv'
prev_date = datetime.now().date() - timedelta(days = 1)

df_hist_perf = historical_performance(hist_perf_file_path, df_weekly_perf, prev_date)


df_hist_perf

,date,total_value,daily_return,cumulative_return
0,2026-01-12,100000.000000,0.000000,0.000000
1,2026-01-13,98858.720724,-0.011413,-0.011413
2,2026-01-14,98186.455750,-0.006800,-0.018135
3,2026-01-15,100014.708930,0.018620,0.000147
